In [ ]:
from autumn.tools.project import get_project
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

from autumn.tools.plots.utils import REF_DATE
from autumn.models.covid_19.constants import BASE_DATETIME
from autumn.tools.calibration.targets import get_target_series
import pandas as pd
from autumn.tools.utils.pretty import pretty_print
from autumn.tools.inputs.covid_au.queries import get_both_vacc_coverage
from summer.utils import ref_times_to_dti

In [ ]:
region = "barwon-south-west"
project = get_project("covid_19", region)

In [ ]:
custom_params = project.param_set.baseline#.update(dict(contact_rate = 0.125))
model = project.run_baseline_model(custom_params)
derived_df = model.get_derived_outputs_df()
scenarios = project.run_scenario_models(model, project.param_set.scenarios)

In [ ]:
sc_derived_df = scenarios[0].get_derived_outputs_df()

In [ ]:
# pretty_print(project.param_set.baseline)

In [ ]:
outputs = ("notifications", "hospital_admissions")

In [ ]:
# pretty_print(custom_params)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 8), sharex="all")
fig.tight_layout(w_pad=1.5, h_pad=3.5)

for i_ax, axis in enumerate(axes.reshape(-1)):
    out = outputs[i_ax]
    target_dates, target_values = get_target_series(project.calibration.targets, REF_DATE, outputs[i_ax])
    axis.plot(ref_times_to_dti(REF_DATE, model.times), derived_df[outputs[i_ax]])
    axis.plot(ref_times_to_dti(REF_DATE, scenarios[0].times), sc_derived_df[outputs[i_ax]])
    axis.scatter(target_dates, target_values, c="k")
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_title(out)

In [ ]:
fig = plt.figure(figsize=(12, 8))
axis = fig.add_subplot()
axis.fill_between(derived_df.index, [0.] * len(derived_df), derived_df["proportion_fully_vaccinated"], label="fully vaccinated")
axis.fill_between(derived_df.index, derived_df["proportion_fully_vaccinated"], derived_df["at_least_one_dose_prop"], label="one dose only")
axis.fill_between(derived_df.index, derived_df["at_least_one_dose_prop"], [1.] * len(derived_df), label="unvaccinated")
vacc_times, vacc_coverage = get_both_vacc_coverage(region.upper().replace("-", "_"))
vacc_dates = ref_times_to_dti(BASE_DATETIME, [int(i) for i in vacc_times])  # Converting numpy ints returned
lagged_dates = ref_times_to_dti(BASE_DATETIME, [int(i) + 14 for i in vacc_times])
axis.plot(vacc_dates, vacc_coverage, color="k", label="actual vaccination")
axis.plot(lagged_dates, vacc_coverage, color="k", linestyle="--", label="lagged for immunity")
axis.tick_params(axis="x", labelrotation=45)
axis.legend()